In [194]:
import os
import json
import numpy
import pandas
import ast
import re
import datetime
import pickle
import difflib

In [36]:
f = open("data/CrimeData/2011_01_assault_clustered.js", "r") 

In [37]:
lines = f.readlines()
f.close()

nHeaderLines = 3
nFooterLines = 3
records = len(lines) - nHeaderLines - nFooterLines


417

In [132]:
def parseLine(jsLine, crimeSet):
    cleanLine = jsLine.replace(" },\n", "")
    cleanLine = cleanLine.replace(" \"type\": \"Feature\", \"properties\": { \"Unnamed: 0\": ", "")
    cleanLine = cleanLine.split(",", 1)
    cleanLine = "{" + cleanLine[1]
    cleanLine = cleanLine.split(", \"geometry")
    cleanLine = cleanLine[0]
    cleanLine = re.sub("\"Offense 2\".*Location\"", "\"Location\"", cleanLine)
    #print(cleanLine)

    try:
        dictLine = ast.literal_eval(cleanLine)
        dictLine["DateTime"] = datetime.datetime.strptime(dictLine["Date"] + " " + dictLine["Time"], "%m\/%d\/%Y %I:%M %p")
        dictLine["CrimeSet"] = crimeSet
        return dictLine
    except ValueError:
        print("ERROR converting string")
        print(cleanLine)
        return "ERROR"


In [134]:
df = pandas.DataFrame()
path = "./data/CrimeData/"
allFiles = os.listdir(path)
nBadRecords = 0

for fileName in allFiles:
    print(fileName)
    f = open(path + fileName, "r") 
    lines = f.readlines()
    f.close()

    nHeaderLines = 3
    nFooterLines = 3
    crimeSet = fileName.split("_")[2]
    nRecords = len(lines) - nHeaderLines - nFooterLines
    print(nRecords)
    records = lines[nHeaderLines + 1:nHeaderLines + nRecords]
    iLine = nHeaderLines + 1
    for record in records:
        dictRecord = parseLine(record, crimeSet)
        if(dictRecord != "ERROR"):
            if(len(df.columns) == 0):
                df = pandas.DataFrame(dictRecord)
            else:
                df = df.append(dictRecord, ignore_index=True) 
        else:
            print("Error on line " + str(iLine))
            nBadRecords += 1;
        iLine += 1

2011_01_assault_clustered.js
411
2011_01_theft_clustered.js
713
2011_02_assault_clustered.js
375
2011_02_theft_clustered.js
603
2011_03_assault_clustered.js
466
2011_03_theft_clustered.js
935
2011_04_assault_clustered.js
438
2011_04_theft_clustered.js
942
2011_05_assault_clustered.js
483
2011_05_theft_clustered.js
922
2011_06_assault_clustered.js
467
2011_06_theft_clustered.js
876
2011_07_assault_clustered.js
423
2011_07_theft_clustered.js
953
2011_08_assault_clustered.js
410
2011_08_theft_clustered.js
805
2011_09_assault_clustered.js
357
2011_09_theft_clustered.js
848
2011_10_assault_clustered.js
364
2011_10_theft_clustered.js
960
2011_11_assault_clustered.js
382
2011_11_theft_clustered.js
840
2011_12_assault_clustered.js
391
2011_12_theft_clustered.js
800
2012_01_assault_clustered.js
438
2012_01_theft_clustered.js
697
2012_02_assault_clustered.js
413
2012_02_theft_clustered.js
680
2012_03_assault_clustered.js
464
2012_03_theft_clustered.js
816
2012_04_assault_clustered.js
458
2012_04

In [139]:
df.head()
pickleFile = open("df.pickle", "wb")
pickle.dump(df, pickleFile)
pickleFile.close()

In [243]:
df = pickle.load( open( "df.pickle", "rb" ))

In [245]:
df = df.drop(['Address,'], axis=1)
df.head()

,Address,CrimeSet,Date,DateTime,Incident Number,Location,Offense 1,Police District,Time,e_clusterK10,...,g_clusterK9,m_clusterK10,m_clusterK2,m_clusterK3,m_clusterK4,m_clusterK5,m_clusterK6,m_clusterK7,m_clusterK8,m_clusterK9
0,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,43.1093,SIMPLE ASSAULT,7.0,11:52 PM,2,...,6,2,1,0,0,0,1,4,0,3
1,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,-87.9528,SIMPLE ASSAULT,7.0,11:52 PM,2,...,6,2,1,0,0,0,1,4,0,3
2,2602 W PORT SUNLIGHT WA,assault,01\/30\/2011,2011-01-30 23:13:00,110300138,"[43.097656, -87.945886]",SIMPLE ASSAULT,5.0,11:13 PM,2,...,3,2,0,0,0,0,1,4,1,3
3,7403 N TEUTONIA AV,assault,01\/29\/2011,2011-01-29 02:37:00,110290012,"[43.152349, -87.958027]",SIMPLE ASSAULT,4.0,02:37 AM,5,...,6,5,1,2,0,0,1,4,0,6
4,5916 N 37TH ST,assault,01\/29\/2011,2011-01-29 02:08:00,110290015,"[43.125092, -87.958524]",SIMPLE ASSAULT,4.0,02:08 AM,5,...,6,5,1,2,0,0,1,4,0,6


In [146]:
## Get aldermanic districts
mprop = pandas.read_csv("./data/Mprop.csv")
mprop.columns

C:\Users\marie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17,26,43,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['TAXKEY', 'SUB_ACCT', 'YR_ASSMT', 'CHK_DIGIT', 'TAX_RATE_CD',
       'PLAT_PAGE', 'HOUSE_NR_LO', 'HOUSE_NR_HI', 'HOUSE_NR_SFX', 'SDIR',
       'STREET', 'STTYPE', 'C_A_CLASS', 'C_A_SYMBOL', 'C_A_LAND', 'C_A_IMPRV',
       'C_A_TOTAL', 'C_A_EXM_TYPE', 'C_A_EXM_LAND', 'C_A_EXM_IMPRV',
       'C_A_EXM_TOTAL', 'P_A_CLASS', 'P_A_SYMBOL', 'P_A_LAND', 'P_A_IMPRV',
       'P_A_TOTAL', 'P_A_EXM_TYPE', 'P_A_EXM_LAND', 'P_A_EXM_IMPRV',
       'P_A_EXM_TOTAL', 'LAST_VALUE_CHG', 'REASON_FOR_CHG', 'CONVEY_DATE',
       'CONVEY_TYPE', 'CONVEY_FEE', 'CHG_NR', 'DIV_ORG', 'DIV_DROP',
       'OWNER_NAME_1', 'OWNER_NAME_2', 'OWNER_NAME_3', 'OWNER_MAIL_ADDR',
       'OWNER_CITY_STATE', 'OWNER_ZIP', 'LAST_NAME_CHG', 'NEIGHBORHOOD',
       'EXM_ACREAGE', 'EXM_PER_CT_LAND', 'EXM_PER_CT_IMPRV', 'BLDG_TYPE',
       'NR_STORIES', 'BASEMENT', 'ATTIC', 'NR_UNITS', 'BLDG_AREA', 'YR_BUILT',
       'SWIM_POOL', 'FIREPLACE', 'AIR_CONDITIONING', 'NR_ROOMS', 'BEDROOMS',
       'BATHS', 'POWDER_ROOMS', 'PARKING_SP

In [192]:
mprop["HOUSE_NR_SFX"] = mprop["HOUSE_NR_SFX"].fillna('')
mprop.loc[mprop["HOUSE_NR_SFX"].str.isalnum(), "HOUSE_NR_SFX"] = "-" + mprop["HOUSE_NR_SFX"]

mprop["FULL_ADDRESS"] = mprop["HOUSE_NR_LO"].map(str) + mprop["HOUSE_NR_SFX"] + " " + mprop["SDIR"] + " " + mprop["STREET"] + " " + mprop["STTYPE"]
mprop.head()

print(mprop['FULL_ADDRESS'].isnull().sum())
mprop['FULL_ADDRESS'] = mprop['FULL_ADDRESS'].fillna('')
print(mprop['FULL_ADDRESS'].isnull().sum())

,TAXKEY,SUB_ACCT,YR_ASSMT,CHK_DIGIT,TAX_RATE_CD,PLAT_PAGE,HOUSE_NR_LO,HOUSE_NR_HI,HOUSE_NR_SFX,SDIR,...,ELEMENT,MSLINK,MAPID,TAX_DELQ,BI_VIOL,RAZE_STATUS,MAP_NAME,MAP_EXT,Unnamed: 92,FULL_ADDRESS
0,19995111,0,2018,4,1,102,9202,9202,,N,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,9202 N 124TH ST
1,19996100,0,2018,4,1,102,12125,12125,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,12125 W COUNTY LINE RD
2,19996210,0,2018,8,1,104,12301,12301,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,12301 W COUNTY LINE RD
3,19997000,0,2018,3,1,101,11715,11715,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,11715 W COUNTY LINE RD
4,19998200,0,2018,1,1,101,11617,11617,,W,...,NaN,NaN,NaN,99999,XXXX,9,NaN,NaN,NaN,11617 W COUNTY LINE RD


In [246]:
#df = df.drop(["Adermanic_District"], axis=1)
df["Aldermanic_District"] = numpy.nan
df.head()
totalFound = 0
totalFuzzy = 0
totalNotFound = 0
maxIndex = 0

,Address,CrimeSet,Date,DateTime,Incident Number,Location,Offense 1,Police District,Time,e_clusterK10,...,m_clusterK10,m_clusterK2,m_clusterK3,m_clusterK4,m_clusterK5,m_clusterK6,m_clusterK7,m_clusterK8,m_clusterK9,Aldermanic_District
0,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,43.1093,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,NaN
1,5048 N 32ND ST,assault,01\/31\/2011,2011-01-31 23:52:00,110320001,-87.9528,SIMPLE ASSAULT,7.0,11:52 PM,2,...,2,1,0,0,0,1,4,0,3,NaN
2,2602 W PORT SUNLIGHT WA,assault,01\/30\/2011,2011-01-30 23:13:00,110300138,"[43.097656, -87.945886]",SIMPLE ASSAULT,5.0,11:13 PM,2,...,2,0,0,0,0,1,4,1,3,NaN
3,7403 N TEUTONIA AV,assault,01\/29\/2011,2011-01-29 02:37:00,110290012,"[43.152349, -87.958027]",SIMPLE ASSAULT,4.0,02:37 AM,5,...,5,1,2,0,0,1,4,0,6,NaN
4,5916 N 37TH ST,assault,01\/29\/2011,2011-01-29 02:08:00,110290015,"[43.125092, -87.958524]",SIMPLE ASSAULT,4.0,02:08 AM,5,...,5,1,2,0,0,1,4,0,6,NaN


In [342]:
#maxIndex = 58035
totalNotFound = 1

In [344]:

for index, row in df.iterrows():
    if index > maxIndex:
        if(index % 100 == 0 and index != 0):
            print("INDEX: " + str(index))
            print("found: " + str(totalFound))
            print("fuzzy: " + str(totalFuzzy))
            print("not found: " + str(totalNotFound))
            print("Null dists: " + str(df["Aldermanic_District"].isnull().sum()))
            pickleFile = open("alder.pickle", "wb")
            pickle.dump(df, pickleFile)
            pickleFile.close()

        if pandas.isnull(row['Address']):
            print('Address is null')
            df.loc[index, 'Aldermanic_District'] = numpy.nan
            totalNotFound += 1

        else:
            rootAddr = row['Address'].split(" #")[0];
            found = mprop['FULL_ADDRESS'].str.contains(rootAddr).any()
            if found == True:
                totalFound += 1
                dist = mprop[mprop['FULL_ADDRESS']==rootAddr]['GEO_ALDER'].tolist()
                if len(dist) == 0:
                    dist = numpy.nan
                else:
                    dist = dist[0]
                df.loc[index, 'Aldermanic_District'] = dist
            else:
                print("    " + rootAddr)
                matches = difflib.get_close_matches(rootAddr, mprop['FULL_ADDRESS'].tolist(), 5)
                #mprop.loc[mprop["FULL_ADDRESS"] == matches, "GEO_ALDER"]
                dist = mprop[mprop["FULL_ADDRESS"].isin(matches)]["GEO_ALDER"].mode()
                #row['Aldermanic_District'] = dist[0]
                #df.iloc[index, 'Aldermanic_District'] = dist[0]
                if len(dist) == 0:
                    df.loc[index, 'Aldermanic_District'] = numpy.nan
                else:
                    df.loc[index, 'Aldermanic_District'] = dist[0]
                totalFuzzy += 1
        maxIndex = index

Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address 

    3780-1 S 43RD ST
    983 W ARTHUR AV
    1216-B W HOPE AV
    5158-A N TEUTONIA AV
    4476 N 29TH ST
INDEX: 59500
found: 42805
fuzzy: 20696
not found: 548
Null dists: 13313
    5059 N 21ST ST
    5240 N 38TH ST
    7215 N 38TH ST
    4507-A W VILLARD AV
    6300 W CAPITOL DR
    5443-E N 92ND ST
    7300 W BECKETT AV
    8925 W MILL RD
    6001 W HAMPTON AV
    4854 N 66TH ST
    8236 W VILLARD AV
    8711 W EGGERT PL
    5258-B N 84TH ST
    4090 N 64TH ST
    7239 W SILVER SPRING DR
    7223 W SILVER SPRING DR
    1830-A N ARLINGTON PL
    2672 N PIERCE ST
    3033-B N BARTLETT AV
    2394 E NORTH AV
    2220-A E NEWBERRY BL
    3400 W JUNEAU AV
    2819 W JUNEAU AV
    1612 W WISCONSIN AV
    2909 W WISCONSIN AV
    3020-A W JUNEAU AV
    1334-B N 22ND ST
    777 N JEFFERSON ST
    1132 N 35TH ST
    4752-A N 77TH CT
    6508 N BEALE ST
    3532 N 85TH ST
    6629 N 105TH ST
    1418 N 9TH ST
    3869-A N 21ST ST
INDEX: 59600
found: 42870
fuzzy: 20731
not found: 548
Null dists:

    3231-A N 46TH ST
    3540 W COURTLAND AV
    3055-B N 48TH ST
    1800 W PIERCE ST
    3400 W BRANTING LA
    2300 W MITCHELL ST
INDEX: 60500
found: 43451
fuzzy: 21050
not found: 548
Null dists: 12326
    2202 S 26TH ST
    2202 S 26TH ST
    1621 W CANAL ST
    6055 N 61ST ST
    5430 N 62ND ST
    8001 N 76TH ST
    7600 W MILL RD
    7630 W DEAN RD
    8936 N 96TH ST
    8553 W FAIRY CHASM DR
    6416 N 76TH ST
    5746 N 64TH ST
    9217 N 75TH ST
    3977 N 70TH ST
    1513 N HAWLEY RD
    3976 N 71ST ST
    8939 W TRIPOLI AV
    2327 W SUNBURY CT
    1636-A S 18TH ST
    919-A W NATIONAL AV
    2561-B S 12TH ST
    2440-B S 15TH ST
    1561-B W MITCHELL ST
    1733-A W WASHINGTON ST
    1420 S 15TH PL
    605 S 1ST ST
    1900 W SCOTT ST
    1804-A W GREENFIELD AV
    1561-A S 2ND ST
    314-A W WASHINGTON ST
    1532-A S UNION ST
    1815 W GRANT ST
    1301 W WHITAKER AV
INDEX: 60600
found: 43518
fuzzy: 21083
not found: 548
Null dists: 12227
    3231-B S 14TH ST
    1300 W 

    5424 N 63RD ST
    6300 W THURSTON AV
    9040 N 85TH ST
    9217 N 75TH ST
    9657 W BEATRICE ST
    7953 N 94TH ST
INDEX: 61600
found: 44200
fuzzy: 21401
not found: 548
Null dists: 11243
    8955 N 96TH ST
    2658-A N 55TH ST
    2357 N 57TH ST
    1 S BREWERS WA
    3016 N 53RD ST
    6341 W KEEFE AVENUE PK
    2438-A N 54TH ST
    227-B N 37TH ST
    3215-B N 53RD ST
    123 S 70TH ST
    3377 S 99TH CT
    8700 W HOWARD AV
    3873 S MINER ST
    2537-B S 15TH ST
    2030-A S 13TH ST
    814 S 11TH ST
    1600 W LAPHAM ST
    1747-B S 2ND ST
    1021-B W GRANT ST
    1713 S 18TH ST
    1208 S 15TH PL
    1024 S 21ST ST
    1322 W WASHINGTON ST
    1709 S 8TH ST
    4576 S 20TH ST
    4576 S 20TH ST
    2458-B S 17TH ST
    5659 S 36TH CT
    2454-A S 6TH ST
    2648 S 10TH ST
    2636 N 48TH ST
    3200 W VINE ST
INDEX: 61700
found: 44268
fuzzy: 21433
not found: 548
Null dists: 11147
    1843-A N 12TH ST
    2140 N 29TH ST
    2652 N 50TH ST
    2768-A N 16TH ST
    2543-A N

    2751 W GREENFIELD AV
    1728 S 25TH ST
    3308 W GREENFIELD AV
    2004 S 19TH ST
INDEX: 62700
found: 44952
fuzzy: 21749
not found: 548
Null dists: 10161
    2626 W BEECHER ST
    905 S 32ND ST
    2433 W BECHER ST
    3724 W BURNHAM ST
    5902 N 65TH ST
    9265-C N 75TH ST
    9102 N 75TH ST
    9253-C N 75TH ST
    8800 N SWAN RD
    9732 W BROWN DEER RD
    8253 N 107TH ST
    8342 N 93RD ST
    9100 W BROWN DEER RD
    5704-B W SILVER SPRING DR
    5746 N 64TH ST
    9666 W BRADLEY RD
    5679 N 60TH ST
    429-A N 40TH ST
    3141 N 50TH ST
    3040 N 55TH ST
    5006-A W VLIET ST
    510 S 76TH ST
    3164A S 37TH ST
    2718 W HOWARD AV
    2467-A S 44TH ST
    1720 S 8TH ST
    1513-A W GREENFIELD AV
    1568 S MUSKEGO AV
    2523 S 15TH ST
    2129 S 15TH ST
    1824-A W GREENFIELD AV
    1226-B S 8TH ST
    2127 S 16TH ST
    2642-B S 15TH PL
    2150 S 13TH ST
    505-C W MAPLE ST
    2509 S 9TH PL
    400 W CANAL ST
    1943 S 5TH PL
    2120 W LAYTON AV
INDEX: 6280

...

    2339-A N BUFFUM ST
    3048-A N 46TH ST
    3014 N 49TH ST
    4750 N 48TH ST
    3235-A N 36TH ST
    3524 N SHERMAN BL
    3247 N 42ND ST
    2808 N 45TH ST
    2124-A W ORCHARD ST
    906-A S 26TH ST
    1801 W CANAL ST
    1558 S 24TH ST
    2216 W FOREST HOME AV
    1618-B S 25TH ST
    925 S 36TH ST
    3731-A W HILDA PL
    1558 S 24TH ST
INDEX: 67100
found: 47939
fuzzy: 23162
not found: 548
Null dists: 5806
    1700 S 29TH ST
    8655 N GRANVILLE RD
    8549 N SERVITE DR
    8569 N GRANVILLE RD
    9909 W FOUNTAIN AV
    9630 W BEATRICE ST
    9253-G N 75TH ST
    9009 N 75TH ST
    100 N 64TH ST
    7306 W BURLEIGH ST
    2639 N 52ND ST
    2479 N 53RD ST
    6730 W KEEFE AVENUE PK
    6730 W KEEFE AVENUE PK
    2525 S 44TH ST
    2351 S 7TH ST
    2400 S 11TH ST
    1211 W ROGERS ST
    1316-A S 7TH ST
    1420-A S 8TH ST
    1834 S 18TH ST
    1515 W LAPHAM BL
    1434 S 9TH ST
    2248-A S 17TH ST
INDEX: 67200
found: 48015
fuzzy: 23186
not found: 548
Null dists: 5706
  

    631 W HARRISON AV
    2779 N 40TH ST
    2333 N 49TH ST
    2568-A N 17TH ST
    2628 W LISBON AV
INDEX: 68200
found: 48696
fuzzy: 23505
not found: 548
Null dists: 4717
    4915 W CENTER ST
    2938 W LISBON AV
    2333 N 49TH ST
    1108 W VINE ST
    2202 N 42ND ST
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address 

Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address is null
Address 

    2056-B S 5TH PL
    1420 W SCOTT ST
    1563-A S 6TH ST
    2150-A S 16TH ST
    1022 W ORCHARD ST
    2482-B S 9TH ST
    1800 W NATIONAL AV
    1328 S 9TH ST
    1500 W HOWARD AV
    2415 S 16TH ST
    1703 W BOLIVAR AV
    4628 S 20TH ST
    3038 W IONA TR
    4529 S 23RD ST
    2803-A S 9TH PL
    200 W ROSEDALE AV
    3476 S 13TH ST
    2550 N 50TH ST
    2845 N 11TH ST
    4220 W LLOYD ST
    2347 N 38TH ST
    2056 N 34TH ST
    2025 N 35TH ST
INDEX: 69400
found: 48931
fuzzy: 23623
not found: 1395
Null dists: 4371
    2651 N 48TH ST
    2566 N 50TH ST
    2534 N 51ST ST
    2402 N 22ND ST
    4220 W LLOYD ST
    5955 N TEUTONIA AV
    6040 N 41ST ST
    6823 N DARIEN ST
    2067 W ATKINSON AV
    6037 N SHERMAN BL
    5062 N 38TH PL
    5904 N SHERMAN BL
    5100 W HAMPTON AV
    5175 N TEUTONIA AV
    5815 W APPLETON AV
    152-A N 63RD ST
    7141 W BURLEIGH ST
    4574 W WELLS ST
    5520 W VLIET ST
    4733 W VLIET ST
    304 N 76TH ST
    6307 W KEEFE AVENUE PK
    2701

    2637 N 14TH ST
    2673-A N 29TH ST
    1125 W RESERVOIR AV
    2463 N 8TH ST
    4959 N 46TH ST
    4311 N 16TH ST
    4330 N 27TH ST
    4420 W FAIRMOUNT AV
    4042 N 19TH PL
    4055 N 12TH ST
    5105 N 35TH ST
    5008 N 22ND ST
    3714 W DOUGLAS AV
    2716 W BOBOLINK AV
    4311 W MONROVIA WA
    4069 N 26TH ST
    4370 N 29TH ST
    5815 W APPLETON AV
    2808 N 56TH ST
INDEX: 70600
found: 49781
fuzzy: 23973
not found: 1395
Null dists: 3191
    3700 W MICHIGAN ST
    5815 W APPLETON AV
    629 N 36TH ST
    5401 W MEINECKE AV
    602 S 67TH ST
    2952 N 54TH ST
    8234 W O CONNOR ST
    6035 W ADLER ST
    1729 N 47TH ST
    2373 N 52ND ST
    2858 N 52ND ST
    2630 N 53RD ST
    6000 W LISBON AV
    2837 N 51ST ST
    5313 W CENTER ST
    5305 W CENTER ST
    2647 N 52ND ST
    624 S 69TH ST
    5815 W APPLETON AV
    3400 S 43RD ST
    7301 W VERONA CT
    7505 W OKLAHOMA AV
    3333 S 27TH ST
    3333 S 27TH ST
    6138 W STACK DR
    3333 S 27TH ST
    3811 W LINCO

In [228]:
a = "7061 N TEUTONIA AV #206"
poss = mprop['FULL_ADDRESS'].tolist();
matches = difflib.get_close_matches(a, poss, 5)
print(matches)
dist = mprop[mprop["FULL_ADDRESS"].isin(matches)]["GEO_ALDER"].mode()
dist[0]

['7061 N TEUTONIA AV', '7501 N TEUTONIA AV', '7026 N TEUTONIA AV', '7017 N TEUTONIA AV', '3706 N TEUTONIA AV']


1.0

In [335]:
print(pandas.isnull(row['Address']))
print(pandas.isnull(rootAddr))

True
False


In [346]:
print(maxIndex)
print(len(df))
print(df["Aldermanic_District"].isnull().sum())
pickleFile = open("alderFULL.pickle", "wb")
pickle.dump(df, pickleFile)
pickleFile.close()

71369
71370
2438
